In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import random
import numpy as np
import pandas as pd
from pydub import AudioSegment
import os
from pathlib import Path

## Extracción de los 30 segundos de cada canción

In [ ]:
"""
def recortar_fragmentos(input_folder, output_folder, duracion_ms=30_000):
    input_folder = Path(input_folder)
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)
    
    for genre_folder in input_folder.iterdir():
        if not genre_folder.is_dir():
            continue
        
        genre_output = output_folder / genre_folder.name
        genre_output.mkdir(exist_ok=True)
        
        for file in genre_folder.glob("*.mp3"):
            try:
                audio = AudioSegment.from_file(file)
                if len(audio) < 105_000:
                    print(f"{file.name} omitido (demasiado corto)")
                    continue

                min_start = 60_000
                max_start = 75_000

                start = random.randint(min_start, max_start)
                fragment = audio[start:start + duracion_ms]
                
                fragment = fragment.set_channels(1).set_frame_rate(22050)
                
                output_path = genre_output / (file.stem + ".wav")
                fragment.export(output_path, format="wav")
                print(f"Guardado: {output_path.name}")
            
            except Exception as e:
                print(f"Error con {file.name}: {e}")

recortar_fragmentos("new_songs", "songs_30s")
"""


Guardado: blues.00100.wav
Guardado: blues.00101.wav
Guardado: blues.00102.wav
Guardado: blues.00103.wav
Guardado: blues.00104.wav
Guardado: blues.00105.wav
Guardado: blues.00106.wav
Guardado: blues.00107.wav
Guardado: blues.00108.wav
Guardado: blues.00109.wav
Guardado: blues.00110.wav
Guardado: blues.00111.wav
Guardado: blues.00112.wav
Guardado: blues.00113.wav
Guardado: blues.00114.wav
Guardado: blues.00115.wav
Guardado: blues.00116.wav
Guardado: blues.00117.wav
Guardado: blues.00118.wav
Guardado: blues.00119.wav
Guardado: blues.00120.wav
Guardado: blues.00121.wav
Guardado: blues.00122.wav
Guardado: blues.00123.wav
Guardado: blues.00124.wav
Guardado: blues.00125.wav
Guardado: blues.00126.wav
Guardado: blues.00127.wav
Guardado: blues.00128.wav
Guardado: blues.00129.wav
Guardado: blues.00130.wav
Guardado: blues.00131.wav
Guardado: blues.00132.wav
Guardado: blues.00133.wav
Guardado: blues.00134.wav
Guardado: blues.00135.wav
Guardado: blues.00136.wav
Guardado: classical.00100.wav
Guardado

## Extracción de características

In [ ]:
def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None, mono=True)

        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        
        rms = librosa.feature.rms(y=y)

        spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)

        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)

        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)

        zcr = librosa.feature.zero_crossing_rate(y)

        harmony = librosa.effects.harmonic(y)
        harmony_mean = np.mean(harmony)
        harmony_var = np.var(harmony)
        
        perceptr = librosa.effects.percussive(y)
        perceptr_mean = np.mean(perceptr)
        perceptr_var = np.var(perceptr)
        
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)

        features = {
            "filename": Path(file_path).name,
            "length": len(y)
        }

        features["chroma_stft_mean"] = np.mean(chroma_stft)
        features["chroma_stft_var"] = np.var(chroma_stft)

        features["rms_mean"] = np.mean(rms)
        features["rms_var"] = np.var(rms)

        features["spectral_centroid_mean"] = np.mean(spec_centroid)
        features["spectral_centroid_var"] = np.var(spec_centroid)

        features["spectral_bandwidth_mean"] = np.mean(spec_bw)
        features["spectral_bandwidth_var"] = np.var(spec_bw)

        features["rolloff_mean"] = np.mean(rolloff)
        features["rolloff_var"] = np.var(rolloff)

        features["zero_crossing_rate_mean"] = np.mean(zcr)
        features["zero_crossing_rate_var"] = np.var(zcr)

        features["harmony_mean"] = harmony_mean
        features["harmony_var"] = harmony_var
        features["perceptr_mean"] = perceptr_mean
        features["perceptr_var"] = perceptr_var

        features["tempo"] = tempo

        for i in range(20):
            features[f"mfcc{i+1}_mean"] = np.mean(mfcc[i])
            features[f"mfcc{i+1}_var"] = np.var(mfcc[i])

        return features

    except Exception as e:
        print(f"Error procesando {file_path}: {e}")
        return None


def process_folder(folder_path):
    folder = Path(folder_path)
    rows = []

    for file in folder.glob("*.wav"):
        feats = extract_features(file)
        if feats:
            feats["label"] = folder.name  
            rows.append(feats)

    return pd.DataFrame(rows)


dfBlues = process_folder("genres_original/blues")
dfClassical = process_folder("genres_original/classical")
dfCountry = process_folder("genres_original/country")
dfDisco = process_folder("genres_original/disco")
dfPop = process_folder("genres_original/pop")
dfRock = process_folder("genres_original/rock")
dfSalsa = process_folder("genres_original/salsa")

df_final = pd.concat([dfBlues, dfClassical, dfCountry, dfDisco, dfPop, dfRock, dfSalsa], ignore_index = True)
df_final

          filename  length  chroma_stft_mean  chroma_stft_var  rms_mean  \
0  blues.00000.wav  661794          0.350129         0.088772  0.130184   
1  blues.00001.wav  661794          0.340849         0.094976  0.095908   
2  blues.00002.wav  661794          0.363538         0.085257  0.175473   
3  blues.00003.wav  661794          0.404854         0.093999  0.141040   
4  blues.00004.wav  661794          0.308526         0.087843  0.091501   

    rms_var  spectral_centroid_mean  spectral_centroid_var  \
0  0.002828             1784.122641          129745.484419   
1  0.002373             1530.261767          375915.508778   
2  0.002751             1552.832481          156471.010904   
3  0.006348             1070.153418          184366.009438   
4  0.002305             1835.128513          343249.495746   

   spectral_bandwidth_mean  spectral_bandwidth_var  ...  mfcc16_var  \
0              2002.412407            85834.410406  ...   52.424534   
1              2038.987608        

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350129,0.088772,0.130184,0.002828,1784.122641,129745.484419,2002.412407,85834.410406,...,52.424534,-1.687854,36.535866,-0.408730,41.603172,-2.302677,55.053654,1.222467,46.941353,blues
1,blues.00001.wav,661794,0.340849,0.094976,0.095908,0.002373,1530.261767,375915.508778,2038.987608,213905.103191,...,55.337963,-0.728403,60.231407,0.296872,48.133217,-0.282430,51.106014,0.530644,45.788696,blues
2,blues.00002.wav,661794,0.363538,0.085257,0.175473,0.002751,1552.832481,156471.010904,1747.754087,76295.413398,...,40.641678,-7.724840,47.629646,-1.819024,52.393597,-3.440458,46.643398,-2.238127,30.653151,blues
3,blues.00003.wav,661794,0.404854,0.093999,0.141040,0.006348,1070.153418,184366.009438,1596.422564,166551.844243,...,44.432903,-3.324069,50.218452,0.636311,37.325726,-0.615968,37.257774,-3.405046,31.965258,blues
4,blues.00004.wav,661794,0.308526,0.087843,0.091501,0.002305,1835.128513,343249.495746,1748.410758,88378.704478,...,85.995193,-5.451786,75.276741,-0.915952,53.633236,-4.408018,62.882492,-11.704385,55.190254,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,rock.00135.wav,661500,0.347061,0.081870,0.195436,0.001550,1698.515510,256796.586882,2040.041680,196895.907984,...,35.782677,-1.405196,30.032005,-3.814633,24.790665,-1.090526,29.271328,-0.163735,47.753235,rock
807,rock.00136.wav,661500,0.370648,0.075609,0.210945,0.001815,2116.250070,359533.846931,2203.355453,282968.221853,...,73.444221,-4.120581,55.364395,-1.342765,60.100986,2.587203,64.869774,1.770604,38.446648,rock
808,rock.00137.wav,661500,0.481612,0.069261,0.269862,0.002897,2685.800548,186107.169174,2669.996117,105792.156387,...,20.931904,-7.325222,20.774944,-1.571330,22.856760,3.632910,21.908964,-3.209435,23.142725,rock
809,rock.00138.wav,661500,0.510498,0.071963,0.253176,0.003111,2680.904183,220202.079329,2702.602516,99163.338392,...,19.748793,-3.716445,21.758575,0.133201,25.211630,3.278928,21.682110,-2.429129,17.545622,rock


In [12]:
df_final_final = df_final.drop(columns=["filename", "length"])
df_final_final

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,0.350129,0.088772,0.130184,0.002828,1784.122641,1.297455e+05,2002.412407,85834.410406,3805.723030,9.012529e+05,...,52.424534,-1.687854,36.535866,-0.408730,41.603172,-2.302677,55.053654,1.222467,46.941353,blues
1,0.340849,0.094976,0.095908,0.002373,1530.261767,3.759155e+05,2038.987608,213905.103191,3550.713616,2.978311e+06,...,55.337963,-0.728403,60.231407,0.296872,48.133217,-0.282430,51.106014,0.530644,45.788696,blues
2,0.363538,0.085257,0.175473,0.002751,1552.832481,1.564710e+05,1747.754087,76295.413398,3042.410115,7.841309e+05,...,40.641678,-7.724840,47.629646,-1.819024,52.393597,-3.440458,46.643398,-2.238127,30.653151,blues
3,0.404854,0.093999,0.141040,0.006348,1070.153418,1.843660e+05,1596.422564,166551.844243,2184.879029,1.493078e+06,...,44.432903,-3.324069,50.218452,0.636311,37.325726,-0.615968,37.257774,-3.405046,31.965258,blues
4,0.308526,0.087843,0.091501,0.002305,1835.128513,3.432495e+05,1748.410758,88378.704478,3579.957471,1.572336e+06,...,85.995193,-5.451786,75.276741,-0.915952,53.633236,-4.408018,62.882492,-11.704385,55.190254,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,0.400696,0.080214,0.187873,0.001510,2048.602858,1.671459e+05,2087.577924,81484.579760,4485.064546,1.037453e+06,...,38.928944,7.061576,40.360695,-8.064692,42.975868,8.189837,76.399803,-6.067052,80.303757,salsa
936,0.316604,0.097497,0.252051,0.004985,3357.173072,5.329062e+05,3271.096494,62490.051643,7663.845324,1.270959e+06,...,56.973175,-6.520112,53.918011,-2.767017,65.810188,-2.362785,79.467216,-6.609877,104.763000,salsa
937,0.323277,0.085396,0.201971,0.004330,2909.971845,1.019119e+06,2855.825142,210959.850087,6176.170987,3.524183e+06,...,57.129147,-3.654883,47.075466,0.785198,60.523502,-6.128338,63.948887,-2.618533,67.443932,salsa
938,0.322024,0.086365,0.207001,0.003961,3076.604739,7.603324e+05,3075.369098,170640.041113,6865.391819,2.886544e+06,...,69.038734,-8.998105,59.124382,0.980226,78.165039,-2.417351,77.145248,1.781945,95.096619,salsa


---

In [16]:
import numpy as np

df2 = df_final_final.copy()

df2 = df2.applymap(
    lambda x: str(x) if isinstance(x, (list, dict, set, np.ndarray)) else x
)

df2.duplicated().sum()

C:\Users\MIGUEL\AppData\Local\Temp\ipykernel_620\1010262308.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2 = df2.applymap(


np.int64(3)

In [17]:
duplicados = df2[df2.duplicated(keep=False)]
duplicados

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
503,0.441018,0.082489,0.122635,0.001560,2418.030527,3.530053e+05,2432.294029,82714.387963,5386.464980,1.413592e+06,...,49.092056,-10.141388,40.081352,4.784342,45.266926,-6.887760,47.371387,-0.988123,41.126026,disco
504,0.441018,0.082489,0.122635,0.001560,2418.030527,3.530053e+05,2432.294029,82714.387963,5386.464980,1.413592e+06,...,49.092056,-10.141388,40.081352,4.784342,45.266926,-6.887760,47.371387,-0.988123,41.126026,disco
594,0.390284,0.099821,0.273408,0.015247,3497.720406,1.376256e+06,3294.829799,157998.372635,7662.173157,3.119360e+06,...,90.872002,-3.867172,83.420746,-2.339084,101.869232,-0.230069,79.912903,2.215880,101.283295,pop
600,0.390284,0.099821,0.273408,0.015247,3497.720406,1.376256e+06,3294.829799,157998.372635,7662.173157,3.119360e+06,...,90.872002,-3.867172,83.420746,-2.339084,101.869232,-0.230069,79.912903,2.215880,101.283295,pop
607,0.447199,0.093037,0.191334,0.007814,3889.142798,1.049768e+06,3271.061089,98629.925836,8055.108317,1.637193e+06,...,111.322189,2.069654,107.434525,1.238651,88.890686,4.586052,95.880180,1.626793,86.331863,pop
611,0.447199,0.093037,0.191334,0.007814,3889.142798,1.049768e+06,3271.061089,98629.925836,8055.108317,1.637193e+06,...,111.322189,2.069654,107.434525,1.238651,88.890686,4.586052,95.880180,1.626793,86.331863,pop


In [18]:
df = df_final_final.drop(index=[504, 600, 611])
df

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,0.350129,0.088772,0.130184,0.002828,1784.122641,1.297455e+05,2002.412407,85834.410406,3805.723030,9.012529e+05,...,52.424534,-1.687854,36.535866,-0.408730,41.603172,-2.302677,55.053654,1.222467,46.941353,blues
1,0.340849,0.094976,0.095908,0.002373,1530.261767,3.759155e+05,2038.987608,213905.103191,3550.713616,2.978311e+06,...,55.337963,-0.728403,60.231407,0.296872,48.133217,-0.282430,51.106014,0.530644,45.788696,blues
2,0.363538,0.085257,0.175473,0.002751,1552.832481,1.564710e+05,1747.754087,76295.413398,3042.410115,7.841309e+05,...,40.641678,-7.724840,47.629646,-1.819024,52.393597,-3.440458,46.643398,-2.238127,30.653151,blues
3,0.404854,0.093999,0.141040,0.006348,1070.153418,1.843660e+05,1596.422564,166551.844243,2184.879029,1.493078e+06,...,44.432903,-3.324069,50.218452,0.636311,37.325726,-0.615968,37.257774,-3.405046,31.965258,blues
4,0.308526,0.087843,0.091501,0.002305,1835.128513,3.432495e+05,1748.410758,88378.704478,3579.957471,1.572336e+06,...,85.995193,-5.451786,75.276741,-0.915952,53.633236,-4.408018,62.882492,-11.704385,55.190254,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,0.400696,0.080214,0.187873,0.001510,2048.602858,1.671459e+05,2087.577924,81484.579760,4485.064546,1.037453e+06,...,38.928944,7.061576,40.360695,-8.064692,42.975868,8.189837,76.399803,-6.067052,80.303757,salsa
936,0.316604,0.097497,0.252051,0.004985,3357.173072,5.329062e+05,3271.096494,62490.051643,7663.845324,1.270959e+06,...,56.973175,-6.520112,53.918011,-2.767017,65.810188,-2.362785,79.467216,-6.609877,104.763000,salsa
937,0.323277,0.085396,0.201971,0.004330,2909.971845,1.019119e+06,2855.825142,210959.850087,6176.170987,3.524183e+06,...,57.129147,-3.654883,47.075466,0.785198,60.523502,-6.128338,63.948887,-2.618533,67.443932,salsa
938,0.322024,0.086365,0.207001,0.003961,3076.604739,7.603324e+05,3075.369098,170640.041113,6865.391819,2.886544e+06,...,69.038734,-8.998105,59.124382,0.980226,78.165039,-2.417351,77.145248,1.781945,95.096619,salsa


In [19]:
df.to_csv("song_features.csv", index=False)